In [53]:
import spacy
import en_core_web_sm
from spacy.pipeline import merge_entities
import re
nlp = en_core_web_sm.load()
nlp.add_pipe(merge_entities)

In [2]:
def process(keywords):
    keywords_hypen = [x.replace(" ", "-") for x in keywords]
    total_keywords = list(set(keywords + keywords_hypen))
    return total_keywords

def is_phrase_in(phrase, text):
    return re.search(r"\b{}(s|es)?\b".format(phrase), text, re.IGNORECASE) is not None

In [3]:
keywords_aadhar = ['Aadhaar', 'UIDAI', 'unique identity number', 'UID', \
            'unique Aadhaar number', 'Unique Identification', \
            'Adhar', 'Aadhar', 'Adhaar', 'Adharcard', 'Aadharcard', \
            'Aadhaarcard', 'Aadhar Card','Aadhar', 'Aadhaar', 'Adhar',\
            'Adharcard', 'Aadharcard', 'Aadhaarcard', 'UIDAI', 'Aadhar Card', \
            '12-digit', 'Enrolment ID', 'Enrollment ID', 'Enrolment Number', \
            'Enrollment Number', 'Personal Identification Number', 'E-Aadhaar', 'UID/EID', \
            'E-Adhar', 'E-Aadhar', 'E-Adhaar', 'Pankaj Kumar', 'uidai.gov.in', \
            'Aadhaar-link', 'Adhaar-link', 'Aadhar-link', 'Adhar-link', \
            'Aadhaar-enable', 'Adhaar-enable', 'Aadhar-enable', 'Adhar-enable'
            ]

keywords_farmers = ['loan waiver', 'loan waivers', 'farmer loan', 'farmer suicide','farmer suicides',\
                    'pest infestation', 'farmer loans','drought','farmer', 'farmers', 'crop insurance',\
                    'Swaminathan Commission', 'National Commission on Farmer', 'kisan', 'agriculture',\
                    'monsoon failure', 'crop failure', 'fertilizers', 'Seeds Corporation', 'crop loss',\
                    'crop losses', 'unseasonal rains', 'irrigation facilities', 'debt traps',\
                   'loan waiver', 'farmer loan', 'farmer suicide', 'pest infestation', 'Swaminathan Commission',\
                    'National Commission on Farmer','kisan', 'monsoon failure', 'crop failure',\
                    'fertilizer', 'Seeds Corporation', 'farmer', 'agricultural', 'crop prices', \
                   'agrarian', 'NCSTC']

keywords_demon = ['Rs 1,000 notes', 'Rs 500 notes', 'lower denomination', 'Rs 500 and Rs 1,000 notes',\
                 'demonetisation', 'denomination note', 'cash withdrawal', 'swipe machine', 'unaccounted money',\
                 'withdrawal limit', 'black money', 'long queue', 'cashless transaction', 'cashless economy',\
                 'demonitis', 'demonitiz', 'swipe machine', 'pos machine', 'fake currency', 'digital payment',\
                 'digital transaction', 'cash transaction', 'cashless economy', 'cash crunch', 'currency switch'\
                 , 'demonetised note', 'cashless transaction', 'note ban', 'currency switch','ATMs', 'now-defunct currency',\
                 'demonitis', 'demonitiz', 'denomination note', 'cash withdrawal', 'swipe machine', 'unaccounted money', 'withdrawal limit', \
                  'pos machine', 'fake currency', 'digital payment', 'digital transaction', 'cash transaction', 'cashless economy',\
                  'black money', 'cash crunch', 'currency switch', 'long queue', 'demonetised note',\
                  'cashless transaction', 'note ban', 'currency switch', 'demonetis', 'demonetiz']

keywords_gst = ['GST', 'Goods and Services Tax', 'Goods & Services Tax', 'Excise Duty',\
                'good and service tax', 'tax reform', 'goods and services tax', 'gst', 'taxpayers',\
               'GST', 'Goods and Services Tax', 'Goods & Services Tax', 'excise duty', \
               'GSTIN', 'CGST', 'SGST', 'IGST', 'Reverse Charge', 'GSTR', 'GSP', 'Suvidha Provider', \
               'HSN']

keywords_tech = ['privacy', 'cashless', 'technology', 'technological', 'innovation', 'software', 'engineering', 'high technology',\
            'technical', 'tech', 'personal data protection']

policies = {"aadhar":process(keywords_aadhar), "demon":process(keywords_demon), "farmers":process(keywords_farmers), "gst":process(keywords_gst), "tech":process(keywords_tech)}

nlp = en_core_web_sm.load()
nlp.add_pipe(merge_entities)

def is_relevant(sent, keywords):
    doc = nlp(sent)
    for chunk in (doc.noun_chunks):
        if chunk.root.dep_!='nsubj' and chunk.root.dep_!='nsubjpass':
            continue
        for word in keywords:
            if word in chunk.text:
                return True
    return False 

In [51]:
files = ['aadhar_all.txt', 'demon_all.txt', 'farmers_all.txt', 'gst_all.txt','tech_all.txt']
idx = 4 # --------------------- indicate policy
f = open('./top-300/' + files[idx], 'r+')
dicti = {'aadhar_all.txt':keywords_aadhar, 
         'demon_all.txt':keywords_demon, 
         'farmers_all.txt':keywords_farmers, 
         'gst_all.txt':keywords_gst,
         'tech_all.txt':keywords_tech}
print(files[idx])
rel_count = 0
total_count = 0
total_keywords = process(dicti[files[idx]])
print(total_keywords)

tech_all.txt
['privacy', 'tech', 'PRIVACY', 'technical', 'TECHNICAL', 'technological', 'engineering', 'TECH', 'cashless', 'ENGINEERING', 'PERSONAL DATA PROTECTION', 'TECHNOLOGICAL', 'high technology', 'innovation', 'personal data protection', 'technology', 'INNOVATION', 'CASHLESS', 'TECHNOLOGY', 'SOFTWARE', 'HIGH TECHNOLOGY', 'software']


In [52]:
f2 = open('./top-300/pobj_relevant_' + files[idx], 'w+')
for line in f:
    if(is_relevant(line.strip(), total_keywords)):
        rel_count += 1
        f2.write(line)
    total_count += 1
f2.close()
print(rel_count, total_count)

180 965


In [48]:
def read_into_string(filename):
    text_file = open(filename, 'r')
    lines = text_file.read().split('\n')
    text_file.close()
    return lines
print(files[idx])
dataset = read_into_string('./top-300/' + files[idx])

gst_all.txt


In [49]:
def supervised_results(X_test, text_clf):
    predicted = text_clf.predict(X_test)
    f = open('./top-300/supervised_relevant_' + files[idx], 'w+')
    for sentence, prediction in zip(X_test, predicted):
        if(prediction == 1):
            f.write(sentence + '\n')
    f.close()
    print(sum(predicted)) # relevant-1, non-relevant-0

In [50]:
import pickle as pkl
supervised_results(dataset, pkl.load(open('./top-300/' + files[idx][:-8] + '-model.pkl', 'rb')))

248


In [33]:
dataset

['deputy chairman of planning commission montek singh ahluwalia told toi that uidai will detail its position on the gpe system to the apex court in an affidavit to be filed shortly.',
 'gpe also said the government was mulling incorporating aadhar numbers in the future resident identity cards of the national population register(npr), which falls under the home ministry.',
 'nandan nilekani , uidai chairman, said the remittance facility demonstrated one of the many uses of gpe.',
 'on nandan nilekani contesting the lok sabha seat from bangalore south constituency, ram said the uidai chairman has expressed his intentions to contest and he told him he would discuss the issue with the congress high command.',
 "share this article related article government opposes jairam's plea against aadhaar bill as money billgovernment opposes jairam's plea against aadhaar bill as money bill the lpg puzzlethe lpg puzzle aadhaar bill is through after opposition scores a few brownie pointsaadhaar bill is 

In [13]:
def check_policy(sent, policies):
    p = []
    words = []
    for policy, keywords in policies.items():
        for word in keywords:
            sentlist = word_tokenize(sent.lower())
            if word.lower() in sentlist:
                words.append(word)
                p.append(policy)
                break
#     if len(p)==0:
#         p.append('tech')
    if len(p)>1:
        print(p)
    return p, words

In [14]:
all_sentences = []
refined_sentences = []
with open('./top-300/all-by-stmnts.txt','r') as file:
    for line in file:
        sentence = line.strip().lower()
        if(len(sentence) > 0):
            all_sentences.append(sentence)
refined_sentences = list(set(all_sentences))

In [15]:
len(all_sentences), len(refined_sentences)

(52579, 50686)

In [16]:
output = open('./top-300/refined-all-by-stmnts.txt','w')
for line in refined_sentences:
    output.write(line + '\n')
output.close()

In [17]:
output = open('./top-300/out-all-by-stmnts.txt','w')
with open('./top-300/refined-all-by-stmnts.txt','r') as file:
    for line in file:
        p, words = check_policy(line, policies)
        p = " ".join(p)
        words = " ".join(words)
        output.write(p+" -- "+words+"\n")
output.close()

['demon', 'gst']
['farmers', 'tech']
['aadhar', 'tech']
['farmers', 'tech']
['gst', 'tech']
['demon', 'gst']
['demon', 'gst']
['farmers', 'tech']
['demon', 'gst']
['farmers', 'tech']
['farmers', 'tech']
['farmers', 'gst']
['farmers', 'tech']
['demon', 'tech']
['aadhar', 'tech']
['aadhar', 'tech']
['aadhar', 'tech']
['demon', 'farmers']
['aadhar', 'tech']
['aadhar', 'farmers']
['farmers', 'tech']
['demon', 'gst']
['demon', 'farmers']
['farmers', 'tech']
['aadhar', 'tech']
['farmers', 'tech']
['aadhar', 'farmers']
['demon', 'tech']
['aadhar', 'gst']
['farmers', 'tech']
['aadhar', 'tech']
['demon', 'farmers']
['farmers', 'tech']
['farmers', 'tech']
['farmers', 'tech']
['gst', 'tech']
['farmers', 'tech']
['aadhar', 'tech']
['demon', 'tech']
['demon', 'farmers']
['demon', 'tech']
['demon', 'gst']
['farmers', 'tech']
['aadhar', 'tech']
['farmers', 'tech']
['farmers', 'tech']
['demon', 'tech']
['demon', 'farmers']
['demon', 'gst']
['demon', 'gst']
['demon', 'farmers']
['aadhar', 'farmers']
['

In [18]:
import itertools

In [19]:
file_a = open('./top-300/aadhar_all.txt','w')
file_d = open('./top-300/demon_all.txt','w')
file_f = open('./top-300/farmers_all.txt','w')
file_g = open('./top-300/gst_all.txt','w')
file_t = open('./top-300/tech_all.txt','w')

In [20]:
with open("./top-300/refined-all-by-stmnts.txt",'r') as file_s, open("./top-300/out-all-by-stmnts.txt",'r') as file_o: 
    for x, y in zip(file_s, file_o):
        y = y.strip().split(" ")
        if len(y)>1:
            print(y)
        for policy in y:
            if policy=="aadhar":
                file_a.write(x)
            if policy=="farmers":
                file_f.write(x)
            if policy=="demon":
                file_d.write(x)
            if policy=="tech":
                file_t.write(x)
            if policy=="gst":
                file_g.write(x)

['farmers', '--', 'farmers']
['farmers', '--', 'farmers']
['farmers', '--', 'farmers']
['demon', '--', 'demonetisation']
['tech', '--', 'engineering']
['gst', '--', 'GST']
['farmers', '--', 'farmers']
['tech', '--', 'privacy']
['farmers', '--', 'agriculture']
['farmers', '--', 'farmers']
['farmers', '--', 'farmers']
['tech', '--', 'technology']
['farmers', '--', 'farmers']
['gst', '--', 'GST']
['tech', '--', 'cashless']
['farmers', '--', 'farmers']
['demon', '--', 'demonetisation']
['demon', '--', 'demonetisation']
['demon', '--', 'demonetisation']
['demon', '--', 'demonetisation']
['aadhar', '--', 'UIDAI']
['tech', '--', 'technology']
['demon', '--', 'demonetisation']
['farmers', '--', 'FERTILIZERS']
['gst', '--', 'GST']
['aadhar', '--', 'Aadhar']
['tech', '--', 'technology']
['demon', '--', 'demonetisation']
['tech', '--', 'technology']
['farmers', '--', 'farmers']
['demon', '--', 'demonetisation']
['farmers', '--', 'farmers']
['farmers', '--', 'farmers']
['gst', '--', 'GST']
['gst',

['tech', '--', 'privacy']
['demon', '--', 'demonetisation']
['farmers', '--', 'farmers']
['gst', '--', 'GST']
['tech', '--', 'innovation']
['farmers', '--', 'agriculture']
['farmers', '--', 'farmers']
['aadhar', '--', 'Aadhar']
['demon', '--', 'demonetisation']
['aadhar', '--', 'Aadhar']
['gst', '--', 'GST']
['farmers', '--', 'AGRICULTURAL']
['tech', '--', 'innovation']
['gst', '--', 'GST']
['farmers', '--', 'FERTILIZER']
['aadhar', '--', 'Aadhar']
['demon', '--', 'ATMS']
['farmers', '--', 'agriculture']
['farmers', '--', 'farmers']
['farmers', '--', 'AGRARIAN']
['gst', '--', 'GST']
['farmers', '--', 'AGRICULTURAL']
['demon', '--', 'demonetisation']
['farmers', '--', 'farmers']
['tech', '--', 'technology']
['farmers', '--', 'farmers']
['aadhar', '--', 'aadhaar']
['tech', '--', 'technology']
['farmers', '--', 'farmers']
['farmers', '--', 'farmers']
['tech', '--', 'technology']
['farmers', '--', 'farmers']
['tech', '--', 'technology']
['farmers', '--', 'AGRICULTURAL']
['gst', '--', 'GST'

['farmers', '--', 'farmers']
['farmers', '--', 'farmers']
['farmers', '--', 'agriculture']
['farmers', '--', 'farmers']
['tech', '--', 'technology']
['farmers', '--', 'farmers']
['tech', '--', 'technology']
['farmers', '--', 'farmers']
['farmers', '--', 'farmers']
['farmers', '--', 'farmers']
['aadhar', '--', 'Aadhar']
['tech', '--', 'technology']
['gst', '--', 'GST']
['farmers', '--', 'farmers']
['farmers', '--', 'agriculture']
['tech', '--', 'technology']
['farmers', '--', 'farmers']
['farmers', '--', 'farmers']
['farmers', '--', 'AGRICULTURAL']
['farmers', '--', 'farmers']
['farmers', '--', 'farmers']
['tech', '--', 'technology']
['farmers', '--', 'farmers']
['tech', '--', 'technology']
['aadhar', 'tech', '--', 'UIDAI', 'technical']
['gst', '--', 'GST']
['farmers', '--', 'farmers']
['farmers', '--', 'farmers']
['farmers', '--', 'farmers']
['tech', '--', 'technology']
['farmers', '--', 'farmers']
['gst', '--', 'GST']
['farmers', '--', 'farmers']
['farmers', '--', 'farmers']
['farmers

In [131]:
import re

def is_phrase_in(phrase, text):
    return re.search(r"\b{}(s|es)?\b".format(phrase), text, re.IGNORECASE) is not None

phrase = "Purple COW"

In [132]:
print(is_phrase_in(phrase, "Purple cow! make the best pets!"))   # False

True


In [128]:
print(is_phrase_in(phrase, "Your purple cow trampled my hedge!"))  # True

False


In [115]:
keywords_aadhar = ['Aadhaar', 'UIDAI', 'unique identity number', 'UID', \
            'unique Aadhaar number', 'Unique Identification', \
            'Adhar', 'Aadhar', 'Adhaar', 'Adharcard', 'Aadharcard', \
            'Aadhaarcard', 'Aadhar Card','Aadhar', 'Aadhaar', 'Adhar',\
            'Adharcard', 'Aadharcard', 'Aadhaarcard', 'UIDAI', 'Aadhar Card', \
            '12-digit', 'Enrolment ID', 'Enrollment ID', 'Enrolment Number', \
            'Enrollment Number', 'Personal Identification Number', 'E-Aadhaar', 'UID/EID', \
            'E-Adhar', 'E-Aadhar', 'E-Adhaar', 'Pankaj Kumar', 'uidai.gov.in', \
            'Aadhaar-link', 'Adhaar-link', 'Aadhar-link', 'Adhar-link', \
            'Aadhaar-enable', 'Adhaar-enable', 'Aadhar-enable', 'Adhar-enable'
            ]

In [118]:
def process(keywords):
    keywords_hypen = [x.replace(" ", "-") for x in keywords]
    total_keywords = list(set(keywords + keywords_hypen))
    return total_keywords

In [119]:
process(keywords_aadhar)

['Aadharcard',
 'E-Adhar',
 'Enrollment Number',
 'Aadhar-Card',
 'Adhar-link',
 'E-Aadhar',
 '12-digit',
 'Personal Identification Number',
 'Enrolment-Number',
 'E-Adhaar',
 'Adhar-enable',
 'Aadhaar',
 'UIDAI',
 'Adhaar',
 'uidai.gov.in',
 'Aadhaarcard',
 'E-Aadhaar',
 'Aadhaar-link',
 'unique Aadhaar number',
 'Adhar',
 'Aadhar',
 'Unique-Identification',
 'Personal-Identification-Number',
 'Adhaar-link',
 'Enrolment Number',
 'Enrolment-ID',
 'Aadhar-link',
 'Enrollment-ID',
 'Adharcard',
 'Aadhar Card',
 'unique-Aadhaar-number',
 'Enrollment-Number',
 'Enrolment ID',
 'UID/EID',
 'Enrollment ID',
 'unique-identity-number',
 'Pankaj Kumar',
 'Unique Identification',
 'Aadhaar-enable',
 'Aadhar-enable',
 'Adhaar-enable',
 'Pankaj-Kumar',
 'UID',
 'unique identity number']